<a href="https://colab.research.google.com/github/JSJeong-me/GPT-Agent/blob/main/LangChain/00-Single_Agent_OpenAI_ko.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LangChain의 Single Agent (OpenAI 버전 · Colab 실습)

이 노트북은 LangChain의 `agents.ipynb` 튜토리얼을 **OpenAI 모델**로 실행하도록 구성한 한국어 실습 예제입니다.

## 목표
- OpenAI의 Chat 모델(`langchain-openai`)을 사용해 **ReAct 스타일 Single Agent**를 LangGraph 프리빌트로 구현
- **Wikipedia**(무키)와 (선택) **Tavily** 검색 툴 연결
- `MemorySaver`로 **세션 기억**을 유지하고, `.stream()`으로 단계별 실행 로그 관찰


In [ ]:
# 0) 필수 라이브러리 설치
!pip -q install langchain langgraph langchain-openai langchain-community wikipedia
# (선택) Tavily 검색을 쓰려면 다음도 설치하세요
!pip -q install langchain-tavily


In [ ]:
import langchain
print(f"LangChain version: {langchain.__version__}")

## 1. 환경 변수 설정 (필수: OpenAI)
- `OPENAI_API_KEY` 를 설정해야 실행됩니다.
- (선택) `TAVILY_API_KEY` 를 설정하면 Tavily 검색 툴도 함께 사용합니다.


In [ ]:
from google.colab import userdata
import openai
import os

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
openai.api_key  = os.environ["OPENAI_API_KEY"]

In [ ]:
# import os
# # 여기에 키를 직접 입력하거나, Colab의 "설정 > 환경변수"에 추가하고 런타임을 재시작하세요.
# # os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"
# # os.environ["TAVILY_API_KEY"] = "YOUR_TAVILY_API_KEY"  # 선택

# if not os.environ.get("OPENAI_API_KEY"):
#     raise RuntimeError("OPENAI_API_KEY 환경변수가 설정되어야 합니다.")
# print("✅ OPENAI_API_KEY 확인 완료")


## 2. 모델 & 도구 준비 (OpenAI 전용)
- OpenAI Chat 모델: `langchain-openai.ChatOpenAI`
- 프리빌트 에이전트: `langgraph.prebuilt.create_react_agent`
- 기본 도구: **Wikipedia** (키 불필요)
- 선택 도구: **Tavily** (키 필요)


In [ ]:
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver
from langchain_openai import ChatOpenAI

from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.tools.wikipedia.tool import WikipediaQueryRun

import uuid # Import the uuid module

# OpenAI 모델 구성 (필요시 모델명 변경 가능: gpt-4o, gpt-4o-mini, gpt-4.1 등)
model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Wikipedia 툴 (무키)
wiki = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(lang="en"))
tools = [wiki]

# (선택) Tavily 검색 툴
try:
    if os.environ.get("TAVILY_API_KEY"):
        from langchain_tavily import TavilySearch
        tools.append(TavilySearch(max_results=2))
except Exception as e:
    print("Tavily 로드 생략:", e)

memory = MemorySaver()
agent = create_react_agent(model, tools, checkpointer=memory)
config = {"configurable": {"thread_id": "openai-" + uuid.uuid4().hex[:8]}}
config

## 3. 스트리밍 실행 (ReAct 프리빌트)
아래 예시는 한국어 질의로 실행하며, 필요 시 도구 호출(위키/탐색)을 자동으로 수행합니다.

> 팁: 스트리밍 중 출력은 에이전트의 추론/도구 호출 단계별 로그를 포함할 수 있습니다.

In [ ]:
prompt = "샌프란시스코의 대표 관광지 3곳을 알려줘. 최신 정보가 필요하면 검색도 활용하고, 한국어로 답변해줘."

for step in agent.stream({"messages": [{"role": "user", "content": prompt}]}, config, stream_mode="values"):
    msg = step["messages"][-1]
    content = msg.content if isinstance(msg.content, str) else str(msg.content)
    print(content)


## 4. 세션 기억 확인 (thread_id 고정)
동일한 `config`(thread_id)를 유지하면 앞선 대화 맥락이 이어집니다.

In [ ]:
follow_up = "가족 여행 기준으로 위 3곳 중 2곳만 추천하고, 추천 이유를 간단히 설명해줘."
result = agent.invoke({"messages": [{"role": "user", "content": follow_up}]}, config)
print(result["messages"][-1].content)


## 5. 실습 과제
1) **커스텀 툴 추가**: 간단한 계산기 함수를 파이썬으로 만들고 LangChain 툴로 래핑하여 에이전트에 등록하세요.

2) **프롬프트 가드레일**: "위키 결과가 부족하면 Tavily 사용 → 그래도 부족하면 사과 후 중단" 규칙을 시스템 프롬프트에 추가해보세요.

3) **한국어 검색 품질 개선**: `WikipediaAPIWrapper(lang="ko")`로 교체해보고 결과를 비교하세요.

4) **LangSmith 트레이싱**(선택): 프롬프트/툴 호출 경로를 시각화해보세요.


## 6. (예시) 커스텀 툴 등록 스니펫
아래 예시를 실행하고 `tools.append(calc_tool)` 후, 에이전트를 재생성하여 활용하세요.

In [ ]:
from langchain.tools import tool

@tool
def add_two_numbers(a: int, b: int) -> int:
    """두 정수의 합을 반환합니다."""
    return a + b

calc_tool = add_two_numbers
print(calc_tool.name, ":", calc_tool.description)


## 참고
- OpenAI Chat 모델: `langchain-openai.ChatOpenAI`
- 프리빌트 ReAct 에이전트: `langgraph.prebuilt.create_react_agent`
- Wikipedia 도구: `langchain_community.tools.wikipedia`
- Tavily 검색: `langchain_tavily.TavilySearch`
- MemorySaver: `langgraph.checkpoint.memory.MemorySaver`
